In [ ]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import random
import os
import sys
sys.path.append('src/')
import nn
import nibabel as nib
from math import floor, ceil
import pipeline
import Unet
import imgaug as ia
from imgaug import augmenters as iaa

In [ ]:
nii_data_dir = "" # path to where NIfTIs are stored
nii_vol_name = "" # volume filename ending in .nii
nii_seg_name = "" # segmentation filename ending in .nii

In [ ]:
nii_vol = nib.load(os.path.join(nii_data_dir, nii_vol_name))
nii_seg = nib.load(os.path.join(nii_data_dir, nii_seg_name))

In [ ]:
nii_vol_arr = nii_vol.get_fdata()
nii_seg_arr = nii_seg.get_fdata()

In [ ]:
nii_vol_arr = np.swapaxes(nii_vol_arr, 0, 2)
nii_seg_arr = np.swapaxes(nii_seg_arr, 0, 2)

In [ ]:
def rotate_nifti(nifti_vol_arr, nifti_seg_arr, degrees):
    # Add explicit channel dimension
    nifti_vol_arr_exp = np.expand_dims(nifti_vol_arr, axis=3)
    nifti_seg_arr_exp = np.expand_dims(nifti_seg_arr, axis=3)
    
    print(nifti_vol_arr_exp.shape, nifti_seg_arr_exp.shape)
    
    rot_vol = iaa.Affine(rotate=degrees, order=1, mode='edge')
    rot_seg = iaa.Affine(rotate=degrees, order=0, mode='edge')
    
    vol_arr_augs = rot_vol.augment_images(nifti_vol_arr_exp)
    seg_arr_augs = rot_seg.augment_images(nifti_seg_arr_exp)
    
    nifti_vol_arr_aug = np.empty(nifti_vol_arr.shape)
    nifti_seg_arr_aug = np.empty(nifti_seg_arr.shape)
    
    num_images = nifti_vol_arr_aug.shape[0]
    
    for i in range(num_images):
        print(i, end=' ')
        nifti_vol_arr_aug[i] = np.squeeze(vol_arr_augs[i])
        nifti_seg_arr_aug[i] = np.squeeze(seg_arr_augs[i])
        
    return nifti_vol_arr_aug, nifti_seg_arr_aug

In [ ]:
degrees = 135 # amount of rotation
nifti_vol_arr_aug, nifti_seg_arr_aug = rotate_nifti(nii_vol_arr, nii_seg_arr, degrees)
print(nifti_vol_arr_aug.shape, nifti_seg_arr_aug.shape)
np.unique(nifti_seg_arr_aug).shape
nii_save_dir = "" # path to save location
pipeline.save_arr_as_nifti(nifti_vol_arr_aug, nii_vol_name, "rotated_volume.nii", nii_data_dir, nii_save_dir)
pipeline.save_arr_as_nifti(nifti_seg_arr_aug, nii_seg_name, "rotated_seg.nii", nii_data_dir, nii_save_dir)